In [1]:
import datasets
import spacy
from collections import Counter
import torch.nn as nn

In [2]:
# Read the dataset using dataset.load_dataset()
dataset = datasets.load_dataset("data/Multi30k_HuggingFace")
train_set, val_set, test_set = dataset['train'], dataset['validation'], dataset['test']
train_set[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

In [3]:
# Use tokenizer from spacy
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

In [ ]:
# Build the dictionary of all tokens first: Vocabulary
vocab_min_freq = 2
sos = '<sos>'
eos = '<eos>'
unk = '<unk>'
pad = '<pad>'
is_lower = True
special_tokens = [unk, pad, sos, eos]
en_token_dict = Counter()
de_token_dict = Counter()
for sentence in train_set:
    en_token_dict.update(sentence['en_tokens'])
    de_token_dict.update(sentence['de_tokens'])

en_token_dict = {k: v for (k, v) in en_token_dict.items() if v > vocab_min_freq}
de_token_dict = {k: v for (k, v) in de_token_dict.items() if v > vocab_min_freq}

en_vocab = special_tokens + list(en_token_dict)
de_vocab = special_tokens + list(de_token_dict)

In [ ]:
def tokenize_example(example, en_nlp, de_nlp, sos_token, eos_token, is_lower):
    en_tokens = [token.text.lower() if is_lower else token.text for token in en_nlp.tokenizer(example['en'])]
    de_tokens = [token.text.lower() if is_lower else token.text for token in de_nlp.tokenizer(example['de'])]

    return {'en_tokens': en_tokens + [eos_token], # input only needs eos token
            'de_tokens': [sos_token] + de_tokens + [eos_token]  # output needs both sos and eos tokens
            }


In [ ]:
fn_kwargs = {
    'en_nlp': en_nlp,
    'de_nlp': de_nlp,
    'sos_token': sos_token,
    'eos_token': eos_token,
    'is_lower': is_lower
}
train_set = train_set.map(tokenize_example, fn_kwargs=fn_kwargs)
train_set = val_set.map(tokenize_example, fn_kwargs=fn_kwargs)
train_set = test_set.map(tokenize_example, fn_kwargs=fn_kwargs)

In [ ]:
print(train_set[0]['en'])
print(train_set[0]['en_tokens'])

In [ ]:
def lookup_tokenids(example, en_vocab, de_vocab):
    en_ids = []
    de_ids = []
    for w in example['en_tokens']:
        en_idx = en_vocab.index(w) if w in en_vocab else 0
        en_ids.append(en_idx)
        
    for w in example['de_tokens']:
        de_idx = de_vocab.index(w) if w in de_vocab else 0
        de_ids.append(de_idx)
    
    return {'en_ids': en_ids,
            'de_ids': de_ids}

In [ ]:
train_set = train_set.map(lookup_tokenids, fn_kwargs={'en_vocab': en_vocab, 'de_vocab': de_vocab})

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, rnn_hidden_dim, rnn_num_layers):
        super().__init__()
        # 1 layer Embedding
        # 2 layers GRU
        # the latent space is the same as the hidden space of the last layer of the GRU
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.encoder = nn.GRU(embedding_dim, rnn_hidden_dim, num_layers=rnn_num_layers, batch_first=True, bias=True)
    
    def forward(self, x):
        x = self.embedding(x)
        # hidden state at the last layer for every word in the sequence:
        #       batch, sequence, hidden_dim
        # final hidden state at every layer
        #       layer, batch, hidden_dim
        x = self.embedding(x)
        state_sequence, state_layer = self.encoder(x)
        return state_sequence, state_layer

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, rnn_hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(embedding_dim, output_dim)
        self.decoder = nn.GRU(rnn_hidden_dim, embedding_dim)
    
    def forward(self, x):
        x = self.embedding(x)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self):
        super().__init__()
        self.decoder = Decoder()
        
    
    def forward(self, x):
